In [1]:
# we are still fixing the reporting class on a non-main branch. 
#This notebook shows we have to fix a few things still.
import sys
sys.path = ['D:/projects/cleanX'] + sys.path

In [2]:
from cleanX.dataset_processing import MLSetup, see_part_potential_bias


In [3]:
# for the sake of the demo we will use nearly the same train and test source 
# i.e. all thousands of images are leaked images between test and train
# setup parameters = train, test, uniqueID, sensitive list 
setup = MLSetup(
    'example_csv.csv',
    'duped1.csv',
    unique_id='X_ray_image_name',
    label_tag='Label',
    sensitive_list=['gender'],
)
report = setup.generate_report()
report.to_ipwidget()

set()
['Label', 'gender']
